# Composite nodes

One of the convenient properties of Forney-style factor graphs (as opposed to regular factor graphs) is that they naturally allow for composability: one can draw a box around part of an FFG and treat this box as a new type of factor node. For example, one can combine a gain (multiplication) node with the addition node in a so-called *composite node*, as depicted in the following graph (Fig. 4.2 from Korl's [A factor graph approach to signal modelling, system identification and filtering](https://www.research-collection.ethz.ch/handle/20.500.11850/82737)):

<img src="./figures/information_filter.png" width="350"/>

Composite nodes are useful for two reasons:

1. Building large graphs becomes more convenient by 'packaging' repetitive parts of the graph as composite nodes.
2. One can define 'shortcut rules' for message updates, which might be more efficient and/or numerically stable than performing vanilla message passing on the internals of the composite node. For example, in the schedule shown above, message (4) is calculated directly from messages (2) and (3). The shortcut rule might exploit the matrix inversion lemma, or involve some optimization algorithm.

To demonstrate the use of composite nodes, in this demo we will build a gain-addition combination that constrains

\begin{align*}
    x_1 = x_0 + b\times u_1\,,
\end{align*}

where `x_0` and `u_1` have Gaussian priors, and b is a constant. We are interested in computing a belief over `x_1`.

We first construct a "flat" graph that represents the gain and addition contraints as two distinct factors, and generate a schedule for inferring a belief for `x_1`. Then, we compare the resulting schedule with a schedule generated on an FFG with a _composite_ gain-addition node. Finally, we show how to register a custom update rule with ForneyLab.

## Schedule generation without composite node

In [1]:
using ForneyLab

# Define factor graph for x1 = x0 + b*u1, where x0 and u1 have Gaussian priors, and b is a constant.
# This is a part of the information filter graph from the introduction.
g = FactorGraph()

b = [1.0; 0.5]

@RV x_0 ~ GaussianMeanVariance(ones(2), eye(2))
@RV u_1 ~ GaussianMeanVariance(1.0, 1.0)
@RV x_1 = x_0 + b*u_1;

In [2]:
flat_schedule = sumProductSchedule(x_1)

draw(g, schedule=flat_schedule) # Inspect the resulting schedule

println(flat_schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 8377486698423437774 -->
 
 8377486698423437774 
 
 clamp_2 
 
<!-- 739085120704230048 -->
 
 739085120704230048 
 
 clamp_1 
 
<!-- 18413387179324013401 -->
 
 18413387179324013401 
 
 clamp_4 
 
<!-- 14848288093254030816 -->
 
 14848288093254030816 
 
 clamp_5 
 
<!-- 334007998810451422 -->
 
 334007998810451422 
 
 𝒩 
 gaussianmeanvariance_2 
 
<!-- 334007998810451422--18413387179324013401 -->
 
 334007998810451422--18413387179324013401 
 
 clamp_4 
 1 out 
 3 v 
 
<!-- 17871170426502282954 -->
 
 17871170426502282954 
 
 clamp_3 
 
<!-- 334007998810451422--17871170426502282954 -->
 
 334007998810451422--17871170426502282954 
 
 clamp_3 
 1 out 
 2 m 
 
<!-- 300892426344513510 -->
 
 300892426344513510 
 
 + 
 addition_1 
 
<!-- 3885668482091198178 -->
 
 3885668482091198178 
 
 × 
 multiplication_1 
 
<!-- 300892426344513510--3885668482091198178 -->
 
 300892426344513510--3885668482091198178 
 
 variable_1 
 1 out 
 (3) 
 3 in2 
 
<!-- 8354590410191311819 -->
 
 8354590410191311819 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 300892426344513510--8354590410191311819 -->
 
 300892426344513510--8354590410191311819 
 
 x_0 
 1 out 
 (1) 
 2 in1 
 
<!-- 3885668482091198178--14848288093254030816 -->
 
 3885668482091198178--14848288093254030816 
 
 clamp_5 
 1 out 
 3 a 
 
<!-- 3885668482091198178--334007998810451422 -->
 
 3885668482091198178--334007998810451422 
 
 u_1 
 1 out 
 (2) 
 2 in1 
 
<!-- 8354590410191311819--8377486698423437774 -->
 
 8354590410191311819--8377486698423437774 
 
 clamp_2 
 1 out 
 3 v 
 
<!-- 8354590410191311819--739085120704230048 -->
 
 8354590410191311819--739085120704230048 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 83407703715118235622 -->
 
 83407703715118235622 
 
<!-- 83407703715118235622--300892426344513510 -->
 
 83407703715118235622--300892426344513510 
 
 x_1 
 1 out 
 (4)

	SPClamp{Multivariate} on Interface 1 (out) of Clamp{Multivariate} clamp_1
	SPClamp{MatrixVariate} on Interface 1 (out) of Clamp{MatrixVariate} clamp_2
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_1
	SPClamp{Multivariate} on Interface 1 (out) of Clamp{Multivariate} clamp_5
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} clamp_3
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} clamp_4
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_2
3.	SPMultiplicationOutVGP on Interface 1 (out) of Multiplication multiplication_1
4.	SPAdditionOutVGG on Interface 1 (out) of Addition addition_1



## Usage of composite nodes in an FFG

Now we 'draw a box' around the multiplication and addition nodes, and create a composite node. We can easily define a composite node using ForneyLab's `@composite` macro.

In [3]:
# Define a composite node for z = x + b*y
@composite GainAddition (z, x, y) begin
    # Specify the 'internal factor graph' of the GainAddion composite node.
    # z, x, and y can be used as if they are existing Variables in this block.
    b = [1.0; 0.5]
    
    @RV z = x + b*y
end

Here, `GainAddition` is the name of the composite node that we're defining. The tuple `(z, x, y)` defines the variables that this node constrains. The order of these variables simultaneously fixes the argument order for the update rules. Now that our custom `GainAddition` composite node is defined, we can use it like any other factor node.

In [4]:
g2 = FactorGraph()

@RV x_0 ~ GaussianMeanVariance(placeholder(:mu_x_0), placeholder(:Sigma_x_0))
@RV u_1 ~ GaussianMeanVariance(
          placeholder(:mu_u_1, dims=(2,)), 
          placeholder(:Sigma_u_1, dims=(2,2)))
@RV x_1 ~ GainAddition(x_0, u_1);

In [5]:
composite_schedule = sumProductSchedule(x_1)

draw(g2, schedule=composite_schedule)

println(composite_schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 16479554071482324147 -->
 
 16479554071482324147 
 
 𝒩 
 gaussianmeanvariance_2 
 
<!-- 73158386941517584 -->
 
 73158386941517584 
 
 placeholder_Sigma_u_1 
 
<!-- 16479554071482324147--73158386941517584 -->
 
 16479554071482324147--73158386941517584 
 
 Sigma_u_1 
 1 out 
 3 v 
 
<!-- 9614072663315595902 -->
 
 9614072663315595902 
 
 placeholder_mu_u_1 
 
<!-- 16479554071482324147--9614072663315595902 -->
 
 16479554071482324147--9614072663315595902 
 
 mu_u_1 
 1 out 
 2 m 
 
<!-- 12072931863720649618 -->
 
 12072931863720649618 
 
 placeholder_Sigma_x_0 
 
<!-- 12116798518647371981 -->
 
 12116798518647371981 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 12116798518647371981--12072931863720649618 -->
 
 12116798518647371981--12072931863720649618 
 
 Sigma_x_0 
 1 out 
 3 v 
 
<!-- 18313896218129764848 -->
 
 18313896218129764848 
 
 placeholder_mu_x_0 
 
<!-- 12116798518647371981--18313896218129764848 -->
 
 12116798518647371981--18313896218129764848 
 
 mu_x_0 
 1 out 
 2 m 
 
<!-- 5199310352204331942 -->
 
 5199310352204331942 
 
 GainAddition 
 gainaddition_1 
 
<!-- 5199310352204331942--16479554071482324147 -->
 
 5199310352204331942--16479554071482324147 
 
 u_1 
 1 out 
 (2) 
 3 y 
 
<!-- 5199310352204331942--12116798518647371981 -->
 
 5199310352204331942--12116798518647371981 
 
 x_0 
 1 out 
 (1) 
 2 x 
 
<!-- 88443853413258242252 -->
 
 88443853413258242252 
 
<!-- 88443853413258242252--5199310352204331942 -->
 
 88443853413258242252--5199310352204331942 
 
 x_1 
 1 z 
 (3)

	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_Sigma_x_0
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_mu_x_0
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_1
	SPClamp{MatrixVariate} on Interface 1 (out) of Clamp{MatrixVariate} placeholder_Sigma_u_1
	SPClamp{Multivariate} on Interface 1 (out) of Clamp{Multivariate} placeholder_mu_u_1
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_2
3.	(INTERNAL SCHEDULE) SPAdditionOutVGG on Interface 1 (z) of GainAddition gainaddition_1



Here, the resulting schedule contains one less message than before, because message (3) directly computes the belief over `x_1` from the prior beliefs. Currently, we have not defined a composite update rule yet and, as a default, message (3) is computed by internal message passing in the composite node. We can inspect this internal schedule.

In [6]:
println(composite_schedule[end].internal_schedule)

	SPClamp{Multivariate} on Interface 1 (out) of Clamp{Multivariate} clamp_1
1.	SPMultiplicationOutVGP on Interface 1 (out) of Multiplication multiplication_1
2.	SPAdditionOutVGG on Interface 1 (out) of Addition addition_1



## Defining custom shortcut rules

If we actually want to use composite nodes to get a different (i.e. more efficient) algorithm, we'll have to specify message update rules that apply specifically to the composite nodes at hand. These rules are sometimes referred to as _shortcut rules_, since they provide a way to shortcut the calculation of internal messages. This rule definition for the composite nodes is analogous to the rule definitions for regular nodes. If we then build a new sum-product algorithm, the custom shortcut rule `SPGainAdditionOutVGG` will be automatically inserted.

In [7]:
@sumProductRule(:node_type     => GainAddition,                                 # our custom composite node
                :outbound_type => Message{GaussianMeanVariance},               # this rule produces a GaussianMeanPrecision msg
                :inbound_types => (Nothing, Message{Gaussian}, Message{Gaussian}), # msg towards first interface, incoming types
                :name          => SPGainAdditionOutVGG)                         # name of the update rule;

In [8]:
shortcut_schedule = sumProductSchedule(x_1)

println(shortcut_schedule)

	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_Sigma_x_0
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_mu_x_0
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_1
	SPClamp{MatrixVariate} on Interface 1 (out) of Clamp{MatrixVariate} placeholder_Sigma_u_1
	SPClamp{Multivariate} on Interface 1 (out) of Clamp{Multivariate} placeholder_mu_u_1
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_2
3.	SPGainAdditionOutVGG on Interface 1 (z) of GainAddition gainaddition_1



In [9]:
algo = sumProductAlgorithm(x_1)
eval(Meta.parse(algo)) # Load algorithm

println(algo) # Inspect the algorithm

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 3))

messages[1] = ruleSPGaussianMeanVarianceOutVPP(nothing, Message(Univariate, PointMass, m=data[:mu_x_0]), Message(Univariate, PointMass, m=data[:Sigma_x_0]))
messages[2] = ruleSPGaussianMeanVarianceOutVPP(nothing, Message(Multivariate, PointMass, m=data[:mu_u_1]), Message(MatrixVariate, PointMass, m=data[:Sigma_u_1]))
messages[3] = ruleSPGainAdditionOutVGG(nothing, messages[1], messages[2])

marginals[:x_1] = messages[3].dist

return marginals

end

end # block


Distribution family of the outgoing message from the composite node "GainAddition" has been specified as GaussianMeanVariance via '@sumProductRule' macro. Mean and variance of this message is computed via the function 'ruleSPGainAdditionOutVGG' implemented below.

In [10]:
function ruleSPGainAdditionOutVGG(msg_out::Nothing,msg_x::Message{F, Vx},msg_y::Message{F, Vy}) where {F<:Gaussian, Vx<:Univariate, Vy<:Multivariate}

    x = convert(ProbabilityDistribution{Vx, GaussianMeanVariance}, msg_x.dist)
    y = convert(ProbabilityDistribution{Vy, GaussianMeanVariance}, msg_y.dist)
    b = [1.0; 0.5]
    Message(Univariate, GaussianMeanVariance, m=x.params[:m]+(reshape(b,1,2)*y.params[:m])[1], 
                v=x.params[:v] + (reshape(b,1,2)*y.params[:v]*reshape(b,2,1))[1])
end

ruleSPGainAdditionOutVGG (generic function with 1 method)

In [11]:
# Prior statistics
mu_x_0 = 1.0
Sigma_x_0 = 1.0
mu_u_1 = ones(2)
Sigma_u_1 = diageye(2)

# Perform inference
#mu_x_0 = deepcopy(mu_x_0)
#Sigma_x_0 = deepcopy(Sigma_x_0)
data = Dict(:mu_x_0 => mu_x_0,
            :Sigma_x_0 => Sigma_x_0,
            :mu_u_1 => mu_u_1,
            :Sigma_u_1 => Sigma_u_1)
    
marginals = step!(data) # Execute the algorithm

Dict{Any,Any} with 1 entry:
  :x_1 => 𝒩(m=2.50, v=2.25)…